# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file


In [3]:
useLogging = False # set to True to get logging information (and hopefully track which LLM is called when)
use_Ollama_For_Inference = True # set to True to use Ollama inference models (and pull at least the gemma:2b model)
use_Ollama_For_Embedding = True # set to True to use Ollama embedddings models (and pull at least the nomic-embed-text:latest model)
use_Pinecone = True #Turn on to use a Pinecone database. Sign up at www.pinecone.io for a free plan (including 5 indexes)
use_Chroma = True #Turn on to use a local Chroma database. Supersedes the use_Pinecone flag above (and turns it off)
use_Test_Data = False # set to True to use LimitedCSVLoader class below and only load the 577th item from the CSV file and test that querying with embeddings work well.

import openai
#Defaults to OpenAI if use_Ollama_For_Inference=False and use_Ollama_For_Inference=False
openai.api_base = inferApiBase = embedApiBase =  "https://api.openai.com/v1"
openai.base_url = inferBaseUrl = embedBaseUrl = "https://api.openai.com"
openai.api_key = inferApiKey = embedApiKey = os.environ['OPENAI_API_KEY']
embeddings_model_name = "text-embedding-ada-002"
embeddings_model_name_short = "ada"
embeddings_vector_size = 1536
infer_model_name = "gpt-3.5-turbo"
llm_platform = "openai"
embed_chunk_size = 1000
embed_overlap = 0


if use_Ollama_For_Inference:
    inferApiBase = "http://localhost:11434/v1"
    inferBaseUrl = "http://localhost:1143"
    inferApiKey = "ollama"
    infer_model_name = "gemma:2b" #you can/should customize this to test different Ollama LLMs. Use the NAME field from `ollama list`


if use_Ollama_For_Embedding:
    llm_platform = "ollama"
    embedApiBase = "http://localhost:11434/v1"
    embedBaseUrl = "http://localhost:1143"
    embedApiKey = "ollama"
    embeddings_model_name = "mxbai-embed-large:latest"
    embeddings_model_name_short = "mxbai"
    embeddings_vector_size = 1024
    embed_chunk_size = 512
    embed_overlap = 10
    #
    #embeddings_model_name = "nomic-embed-text:latest" #you can/should customize this to test different Ollama LLMs. Use the NAME field from `ollama list`
    #embeddings_model_name_short = "nomic"
    #embeddings_vector_size = 768
    #embed_chunk_size = 8192
    #embed_overlap = 0

print('Embed API Key:', embedApiKey)
print('Infer API Key:', inferApiKey)
print('Embed API Base:', embedApiBase)
print('Infer API Base:', inferApiBase)
print('Embeddings Model:', embeddings_model_name)
print('Inference Model:', infer_model_name)


index_prefix = "langchain-deeplearningai-" + embeddings_model_name_short + "-"
if use_Test_Data:
    index_prefix+='s-'
rag_index_name = index_prefix + llm_platform
print('RAG Index Name:', rag_index_name)

if use_Chroma:
    print('using Chroma Vector database')
    import chromadb
    use_Pinecone = False
    storage_path = os.environ.get('CHROMA_STORAGE_PATH')
    if storage_path is None:
        raise ValueError('CHROMA_STORAGE_PATH environment variable is not set')
    

elif use_Pinecone:

    print('using Pinecone Vector database')
    from pinecone import Pinecone
    from langchain_pinecone import PineconeVectorStore
    from tqdm.autonotebook import tqdm

    PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
    PINECONE_ENV = os.environ.get("PINECONE_ENV", "PINECONE_ENV")

    if PINECONE_API_KEY is None:
        raise ValueError("PINECONE_API_KEY environment variable not set.")
        # Name our index on Pineconeopenai.api_key

    # Init pinecone
    pc = Pinecone(
        api_key=PINECONE_API_KEY,
        source_tag="langchain-deeplearningai"
    )
else:
    print('using In Memory Vector database')

from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings


if use_Ollama_For_Embedding:
    embeddings_model = OllamaEmbeddings(model=embeddings_model_name, embed_instruction='', query_instruction='')
    #embeddings_model = OllamaEmbeddings(model=embeddings_model_name)
else:
    embeddings_model = OpenAIEmbeddings(model=embeddings_model_name)

if useLogging:
    import logging
    import requests 


    logging.basicConfig(level=logging.DEBUG,
                        format='%(asctime)s - %(levelname)s - %(filename)s:%(funcName)s:%(lineno)d - %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')
    requests.packages.urllib3.add_stderr_logger()
    OLLAMA_DEBUG=1
else:
    import logging
    print('Disabling logging')
    OLLAMA_DEBUG=0
    logging.basicConfig(level=logging.CRITICAL)

Embed API Key: ollama
Infer API Key: ollama
Embed API Base: http://localhost:11434/v1
Infer API Base: http://localhost:11434/v1
Embeddings Model: mxbai-embed-large:latest
Inference Model: gemma:2b
RAG Index Name: langchain-deeplearningai-mxbai-ollama
using Chroma Vector database
Disabling logging


## Built-in LangChain tools

In [2]:
#!pip install -U wikipedia

In [4]:
from langchain.agents.react.agent import create_react_agent
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain_openai import ChatOpenAI

In [21]:
llm = ChatOpenAI(temperature=0)
infer_model_name = 'llama3:8b'
llm = ChatOpenAI(temperature = 0.0, base_url=inferApiBase, model=infer_model_name)

In [16]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [ ]:
agent= create_react_agent(
    tools, 
    llm,
    prompt=)

In [23]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [24]:
agent.invoke("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Could not parse LLM output: Thought: To find 25% of 300, I need to calculate a quarter of 300.

Action:
```
{
  "action": Calculator,
  "action_input": "300 / 4"
}
```


Observation: Invalid or incomplete response
Thought:Could not parse LLM output: I apologize for the confusion earlier! Let's start fresh.

Thought: To find 25% of 300, I need to calculate a quarter of 300.

Action:
```
{
  "action": Calculator,
  "action_input": "300 / 4"
}
```


Observation: Invalid or incomplete response
Thought:Final Answer: The 25% of 300 is 75.

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': 'The 25% of 300 is 75.'}

In [25]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent.invoke(question) 



> Entering new AgentExecutor chain...
Thought: I need to find information about Tom M. Mitchell's book.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```



h:\Users\Raphael\OneDrive\Perso\Technical\AI\Training\GenAI\LangChain-for-LLM-Application-Development\venv\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file h:\Users\Raphael\OneDrive\Perso\Technical\AI\Training\GenAI\LangChain-for-LLM-Application-Development\venv\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Tom Mitchell (Australian footballer)
Summary: Thomas Mitchell (born 31 May 1993) is a professional Australian rules footb

## Python Agent

In [8]:
infer_model_name = 'phi3:3.8b'
llm = ChatOpenAI(temperature = 0.0, base_url=inferApiBase, model=infer_model_name)
#llm = ChatOpenAI(temperature = 0.0)
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True,
    handle_parsing_errors=True,
)

In [6]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [9]:
agent.invoke(f"""Sort these customers by \
last name and then first name \
and print  the output: {customer_list}""") 



> Entering new AgentExecutor chain...
 Thought: I need to sort the list of customers by last name and then first name.
Action: Python_REPL
Action Input: 
```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[0], x[1]))
print(sorted_customers)
```
Observation: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]

Thought: Final Answer: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]

> Finished chain.


{'input': "Sort these customers by last name and then first name and print  the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
 'output': "[['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]"}

#### View detailed outputs of the chains

In [10]:
import langchain
langchain.debug=True
agent.invoke(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an

## Define your own tool

In [13]:
#!pip install DateTime

In [11]:
from langchain.agents import tool
from datetime import date

In [17]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [18]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

h:\Users\Raphael\OneDrive\Perso\Technical\AI\Training\GenAI\LangChain-for-LLM-Application-Development\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [19]:
try:
    result = agent.invoke("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
Could not parse LLM output:  {
  "action": "time",
  "action_input": ""
}

Thought: I need to find out today's date using the time tool provided.

Observation: Invalid or incomplete response
Thought: Since the previous attempt did not yield a valid result, let's try again with proper formatting and ensure we input an empty string as required by the tool specification.

```
{
  "action": "time",
  "action_input": ""
}
```


Observation: 2024-06-10
Thought: I now know today's date is June 10, 2024.

Final Answer: Today's date is June 10, 2024.

> Finished chain.
